## In this file we will evaluate whether, the extracted indexes (rep frames) are enough to use with deep learning in terms of training and evaluation


In [ ]:
"""
Steps:
1. Create the indexes
2. 1 instance of the network is trained using the whole data with augmentation
3. 1 instance of the network is trained using rep frame data with augmentation
4. Compare training speed
5. Compare training loss
6. Compare mAP
"""

In [ ]:
import os
os.getcwd()
%pylab inline
%load_ext autoreload
%autoreload 2

import os
import sys
sys.argv=['']
sys.path.append('../../')


import numpy as np
#import utils.helpers as helpers
import utils as helpers
from loaders.uadetrac_loader import UADetracLoader
from eva_storage.preprocessingModule import PreprocessingModule
from eva_storage.UNet import UNet
from eva_storage.clusterModule import ClusterModule
from filters.minimum_filter import FilterMinimum
from eva_storage.evaluation.evaluate_compression import *

In [ ]:
# load the data  
loader = UADetracLoader()
images = loader.load_cached_images()
labels = loader.load_cached_labels()
video_start_indices = loader.get_video_start_indices()

In [ ]:
# run background subtraction
pm = PreprocessingModule()
seg_images = pm.run(images,video_start_indices)
print(seg_images.shape)

In [ ]:
# train the main network
unet = UNet()
unet.train(images, seg_images, epoch = 100)


In [ ]:
# Generate the compressed / segmented images
unet_compressed_images, unet_segmented_images = unet.execute()

In [ ]:
# Create clusters
cm = ClusterModule()
image_cluster_labels = cm.run(unet_compressed_images)

In [ ]:
# Generate binary labels
## within labels['vehicle'] there are ['car', 'others', 'van', 'bus']

car_labels = helpers.generateBinaryLabels(labels['vehicle'])
other_labels = helpers.generateBinaryLabels(labels['vehicle'], label_of_interest = 'others')
van_labels = helpers.generateBinaryLabels(labels['vehicle'], 'van')
bus_labels = helpers.generateBinaryLabels(labels['vehicle'], 'bus')